Rag From Scratch:Part 1 -4

In [ ]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromdb langchin

LangSmith

In [4]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = ''

os.environ['OPENAI_API_KEY'] = ''

### Part 1: Overiew

In [18]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader 
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

## indexing ##
loader = WebBaseLoader(
    web_paths = ("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs = dict(
        parse_only = bs4.SoupStrainer(
            class_ = ("post-content","post-title","post-header")
        )      
    ),
) 
docs = loader.load()
## Split docs ##
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
## convert to embed ##
vectorstore = Chroma.from_documents(documents=splits,embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

### Retrievl as Generation ###
## get Prompt
prompt = hub.pull("rlm/rag-prompt")

## LLM model
llm = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0)

## Post processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

## Chai
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt 
    | llm
    | StrOutputParser()
)

## put question
rag_chain.invoke("What is task Decomposition?")

"Task Decomposition is a technique used to break down complex tasks into smaller and simpler steps. It involves transforming big tasks into multiple manageable tasks to aid in the interpretation of the model's thinking process. Task decomposition can be achieved through various methods such as prompting techniques, task-specific instructions, or human inputs."

### Part 2: Indexing

In [21]:
## Documents
question = "What kinds of pets do I like?"
document = "My favorite pet is a cat"

Count tokens

In [ ]:
import tiktoken

def num_token_from_string(string: str, encoding_name: str) -> int:
    """Return the number of tokens in a text string."""
    encoding = tiktoken.get_encoding( encoding_name  )
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_token_from_string(question, "cl100k_base")    

Text embedding model

In [ ]:
from langchain_openai import OpenAIEmbeddings
emdb = OpenAIEmbeddings()
query_result = emdb.embed_query(question)
document_result = emdb.embed_query(document)
len(query_result)

Cosine similarity is recommended for OpenAi embedings

In [ ]:
import numpy as np
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print("cosine similarity:",similarity)

Document Loaders

In [27]:
### indexing ###
# load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths = ("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs = dict(
        parse_only=bs4.SoupStrainer(
            class_ = ("post-content","post-title","post-header")
        )
    ),
)
blog_docs = loader.load()

splitter